In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from tqdm import *

In [2]:
def preprocessing(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    processed_img = cv2.GaussianBlur(gray, (15,15), 0)
    return processed_img

#Function to find features in the two imagess
def feature_detection(img1, img2):
    processed_img1, processed_img2 = preprocessing(img1), preprocessing(img2)
    sift = cv2.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(processed_img1, None)
    kp2, desc2 = sift.detectAndCompute(processed_img2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)
    
    #Extracting good matches
    good = []
    for m, n in matches:
        if m.distance < 0.2*n.distance:
            good.append([m])
    
    left_pts  = np.float32([kp1[m[0].queryIdx].pt for m in good])
    right_pts  = np.float32([kp2[m[0].trainIdx].pt for m in good])
    return left_pts, right_pts


In [3]:
def fundamental_matrix(left_pts, right_pts):
    A = []
    
    for i in range(8):
        src_x, src_y  = left_pts[i]
        dst_x, dst_y = right_pts[i]
        A_row = np.asarray([dst_x*src_x, dst_x*src_y, dst_x, dst_y*src_x, dst_y*src_y, dst_y, src_x, src_y, 1])
        A.append(A_row)
        
    A = np.asarray(A)
    U, S, V = np.linalg.svd(A)
    F = np.reshape(V[-1, :], (3, 3))
    u, s, vt = np.linalg.svd(F)
    s = np.diag(s)
    s[2, 2] = 0
    F = u @ (s @ vt)
    F = F / F[-1,-1]
    F = (abs(F) > (10**(-3))) * F
    return F

#Ransac function to find the best Fundamental Matrix that satisfies the epipolar constraint
def ransac(src_pts, dst_pts, iterations, threshold):
    number_of_points = len(src_pts)
    max_inlier_count = 0
    
    for i in range(iterations):
        inlier = 0
        random_indices = np.random.choice(number_of_points, size = 8, replace=False)
        random_src_pts = src_pts[random_indices, :]
        random_dst_pts = dst_pts[random_indices, :]
        
        F = fundamental_matrix(random_src_pts, random_dst_pts)

        for i in range(number_of_points):
            a, b = dst_pts[i], src_pts[i]
            a, b = np.append(a, 1).reshape((3, 1)), np.append(b, 1).reshape((3, 1))
            c = (a.T @ F) @ b
            
            if abs(c) <= threshold:
                inlier+=1
            
        if inlier > max_inlier_count:
            max_inlier_count = inlier
            best_F = F          
    return best_F